In [1]:
import numpy as np
import cv2
import os
import pickle
import numpy as np
from keras.utils import np_utils


In [2]:
label_dict={"with_mask":0,"without_mask":1}
categories=["with_mask","without_mask"]

In [3]:
data_path="C:\\Users\\devan\\Desktop\\Data Set"

In [4]:
data=[]
target=[]

In [5]:
for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
    for img in img_names:
        img_path=os.path.join(folder_path,img)
        img=cv2.imread(img_path)
        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            resized=cv2.resize(gray,(100,100))
            data.append(resized)
            target.append(label_dict[category])
        except:
            pass

In [6]:
data=np.array(data)
data=data/255
data.shape

(1376, 100, 100)

In [7]:
data=np.reshape(data,(1376,100,100,1))

In [8]:
target=np.array(target)
new_target=np_utils.to_categorical(target)

In [9]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from sklearn.model_selection import train_test_split

In [10]:
model=Sequential()
model.add(Conv2D(200,(3,3),activation="relu",input_shape=data.shape[1:]))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(100,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(50,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation="relu"))
model.add(Dense(2,activation="softmax"))

In [11]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [12]:
train_data,test_data,train_target,test_target=train_test_split(data,new_target,test_size=0.1)

In [31]:
model.fit(train_data,train_target,epochs=20)

Epoch 1/20
39/39 [==============================] - 44s 1s/step - loss: 0.0299 - accuracy: 0.9871
Epoch 2/20
39/39 [==============================] - 41s 1s/step - loss: 0.0265 - accuracy: 0.9927
Epoch 3/20
39/39 [==============================] - 41s 1s/step - loss: 0.0290 - accuracy: 0.9887
Epoch 4/20
39/39 [==============================] - 41s 1s/step - loss: 0.0372 - accuracy: 0.9863
Epoch 5/20
39/39 [==============================] - 39s 1s/step - loss: 0.0179 - accuracy: 0.9927
Epoch 6/20
39/39 [==============================] - 60s 2s/step - loss: 0.0103 - accuracy: 0.9968
Epoch 7/20
39/39 [==============================] - 93s 2s/step - loss: 0.0139 - accuracy: 0.9968
Epoch 8/20
39/39 [==============================] - 48s 1s/step - loss: 0.0329 - accuracy: 0.9871
Epoch 9/20
39/39 [==============================] - 37s 936ms/step - loss: 0.0176 - accuracy: 0.9960
Epoch 10/20
39/39 [==============================] - 38s 976ms/step - loss: 0.0150 - accuracy: 0.9952
Epoch 11/20
3

In [38]:
frameWidth= 640
frameHeight = 480
brightness = 180
threshold = 0.75
font = cv2.FONT_HERSHEY_SIMPLEX

In [39]:
cap = cv2.VideoCapture(0)

In [40]:
def grayscale(img):
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return img
def equalizer(img):
    img=cv2.equalizeHist(img)
    return img
def preprocessing(img):
    img=grayscale(img)
    img=equalizer(img)
    img=img/255
    return img
def getclass(classno):
    if classno==0: return "Mask is weared"
    elif classno==1: return "Mask is not weared"

In [41]:
face=cv2.CascadeClassifier("C:\AI\AI\haarcascades\haarcascades\haarcascade_frontalface_default.xml")

In [42]:
while (1):
    success,imgO=cap.read()
    img=np.asarray(imgO)
    img=cv2.resize(img,(100,100))
    img=preprocessing(img)
    cv2.imshow("Processed Image",img)
    img=img.reshape(1,100,100,1)
    cv2.putText(imgO,"Class: ",(20,35),font,0.75,(255,0,0),2,cv2.LINE_AA)
    cv2.putText(imgO,"Probability: ",(20,75),font,0.75,(255,0,0),2,cv2.LINE_AA)
    gray=grayscale(imgO)
    faces=face.detectMultiScale(gray,1.1,10)
    predictions=model.predict(img)
    classIndex=model.predict_classes(img)
    probabilityValue=np.amax(predictions)
    if probabilityValue>0.75:
        cv2.putText(imgO,str(classIndex) + " "+ str(getclass(classIndex)),(120,35),font,0.75,(255,0,0),2,cv2.LINE_AA)
        cv2.putText(imgO,str(round(probabilityValue*100,2)) + "%",(120,100),font,0.75,(255,0,),2,cv2.LINE_AA)
    for(x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),1)
    cv2.imshow("Result",imgO)
    if(cv2.waitKey(1) & 0xFF==ord('q')):
        cap.release()
        cv2.destroyAllWindows()

TypeError: Expected Ptr<cv::UMat> for argument 'src'